### Import Packages

In [1]:
import cv2
import cvzone
import numpy as np
from cvzone.SelfiSegmentationModule import SelfiSegmentation

### Import video 

#### To get video directly from lap Cam

In [2]:
vid=cv2.VideoCapture(0)
vid.set(3,640)#width
vid.set(4,480)#height



True

In [3]:
#To input a video
#vid = cv2.VideoCapture("video.mp4")
# Readjust background according to size of uploades video

### Import Background Image

In [7]:
bg_img=cv2.imread("./dataset/img.jpg")
bg_img = cv2.resize(bg_img, (640, 480))

### Remove Background of video

In [8]:
seg = SelfiSegmentation()

while True:
    _, video = vid.read()
    
    if video is None:
        break
        
    vid_rmbg = seg.removeBG(video, bg_img, threshold=0.4)
    cv2.imshow("Video", vid_rmbg)
    
    if cv2.waitKey(1) == ord("x"):
        break

vid.release()
cv2.destroyAllWindows()

### Optical flow

#### Open video capture

In [9]:
cap = cv2.VideoCapture("./dataset/video.mp4")

#### Get first frame and convert to grayscale


In [10]:
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)# Color img to gray scale img


#### Create HSV output image


In [11]:
hsv = np.zeros_like(prev_frame) #same shape as the prev_frame image, but with all elements initialized to zero
hsv[:,:,1] = 255 #Saturation second channel set to maximum
hsv

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

#### Looping through each Frame

In [12]:
while True:
    # Captur the new frame and convert to grayscale
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the optical flow using Farneback algorithm
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Convert flow to polar coordinates
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[:,:,0] = angle * 180 / np.pi / 2 # First channel containing angle
    hsv[:,:,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) # Third channel containing normalized magnitude values
    rgb = cv2.cvtColor(cv2.merge([hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]]), cv2.COLOR_HSV2BGR)# Merged back into BGR 
    
    # Display the resulting flow as a colored image
    cv2.imshow('Optical Flow', rgb)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # Update previous frame
    prev_gray = gray

# Release video capture and close window
cap.release()
cv2.destroyAllWindows()
